<a href="https://colab.research.google.com/github/thomasdevasia/deep-image-prior/blob/master/denoise_sar.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!git clone https://github.com/thomasdevasia/deep-image-prior
# !mv deep-image-prior/* ./
%cd deep-image-prior

Cloning into 'deep-image-prior'...
remote: Enumerating objects: 393, done.
remote: Counting objects: 100% (104/104), done.
remote: Compressing objects: 100% (63/63), done.
remote: Total 393 (delta 48), reused 85 (delta 40), pack-reused 289
Receiving objects: 100% (393/393), 123.26 MiB | 24.71 MiB/s, done.
Resolving deltas: 100% (203/203), done.
/content/deep-image-prior


In [2]:
from __future__ import print_function
import matplotlib.pyplot as plt
%matplotlib inline

import os
#os.environ['CUDA_VISIBLE_DEVICES'] = '3'

import numpy as np
from models import *

import torch
import torch.optim

# from skimage.measure import compare_psnr
from skimage.metrics import peak_signal_noise_ratio as compare_psnr
from utils.denoising_utils import *

import scipy.io as sio

torch.backends.cudnn.enabled = True
torch.backends.cudnn.benchmark =True
dtype = torch.cuda.FloatTensor

imsize =-1
PLOT = True
sigma = 25
sigma_ = sigma/255.

In [3]:
# png SAR image
fnamep = 'data/denoising/SAR3.png'
# .mat files
# fname = 'data/denoising/CL7.mat'
fname = 'data/denoising/CL8.mat'
# fname = 'data/denoising/CL9.mat'

# Load and prepare Image

In [4]:
# reading mat files
test = sio.loadmat(fname)
intImage = test['x2']
temp = pil_to_np(intImage)
img_pil = np_to_pil(temp)
img_pil = crop_image(img_pil, d=32)
# img_np = 255*pil_to_np(img_pil)
img_np = pil_to_np(img_pil)
# img_np

In [5]:
img_np.shape

(1, 672, 672)

In [6]:
# img_noisy_pil = crop_image(get_image(fname, imsize)[0], d=32)
# img_noisy_np = pil_to_np(img_noisy_pil)

# img_pil = img_noisy_pil
# img_np = img_noisy_np

# plot_image_grid([img_np], 4, 5);

In [7]:
# im = crop_image(get_image(fname, imsize)[0], d=128)
# im.size

In [8]:
# # reading png files
# # img_pil = get_image(fname, imsize)[0]
# img_pil = crop_image(get_image(fnamep, imsize)[0], d=32)
# # converting to numpy
# img_np = pil_to_np(img_pil)
# # img_np

In [9]:
# # # adding speckle noise
# def get_noisy_img(img_np, sigma):
#   """Adds noise to an image.
    
#   Args: 
#       img_np: image, np.array with values from 0 to 1
#       sigma: std of the noise
#   """

#   np.random.seed(42)

#   noise = np.random.normal(loc=1, scale=sigma_, size=img_np.shape)

#   # multiplicative noise for speckle noise
#   # imgNoise = img_np + (img_np * noise)
#   imgNoise = np.log(img_np * noise)

#   img_noisy_np = np.clip(abs(imgNoise), 0, 1).astype(np.float32)
    
#   img_noisy_pil = np_to_pil(img_noisy_np)

#   return img_noisy_pil, img_noisy_np

In [10]:
# correct way
def generateAddSpeckle(img_np, L):

    '''

    Creates multiplicative speckled image with number of looks L. L increases lower noise effect, L is lower noise is strong. 

    '''

    sum_ = 0

    for i in range(L):

        seq1 = np.random.randn(img_np.shape[1], img_np.shape[2])

        seq2 = np.random.randn(img_np.shape[1], img_np.shape[2])

        temp = np.multiply(seq1, seq1) + np.multiply(seq2, seq2)

        sum_ = sum_ + temp/2

        #print(i)

    speckle_ = np.multiply(sum_,1/L)

    img_noisy_np = np.multiply(img_np, speckle_)

    img_noisy_np = np.clip(img_noisy_np, 0, 1).astype(np.float32)

    img_noisy_pil = np_to_pil(img_noisy_np)

    return img_noisy_pil, img_noisy_np

In [37]:
# with ground truth available
# img_noisy_pil, img_noisy_np = get_noisy_img(img_np, sigma_)
img_noisy_pil, img_noisy_np = generateAddSpeckle(img_np, 10)
plot_image_grid([img_np, img_noisy_np],4,6)
(img_np.shape, img_noisy_np.shape)

In [12]:
# when ground truth not available
# img_noisy_pil = img_pil
# img_noisy_np = img_np

In [13]:
# img_pil
# ar = np.array(img_pil)
# npImg = ar.transpose(2,0,1).astype(np.float32)/255
# img_noisy_np

In [14]:
# noise = np.random.normal(scale=sigma, size=img_np.shape)
# img_noisy_np = np.clip((img_np * noise),0,1).astype(np.float32)
# img_noisy_pil = np_to_pil(img_noisy_np)
# img_noisy_np

# plot_image_grid
# plot_image_grid([img_noisy_np],4,6)
# img_pil

In [15]:
# # from torch.cuda.random import seed
# # checking for the noise to add
# # np.log(img_np * noise)
# # img_np
# np.random.seed(42)
# # noise = np.random.normal(scale=sigma_, size=img_np.shape)
# noise = np.random.uniform(0, 1, img_np.shape)

# print(img_np.shape, noise.shape)
# # imgNoise = img_np + noise
# imgNoise = np.log(img_np + noise)
# # imgNoise = img_np + (img_np * noise)
# res = np.clip(imgNoise, 0, 1).astype(np.float32)
# plot_image_grid([res],4,6)

In [16]:
# # add noise
# img_noisy_np = get_noisy_img(img_np, sigma_)

In [17]:
# plot_image_grid([img_np, img_noisy_np], 4, 6);
# n_channels = max(x.shape[0] for x in [img_np, img_noisy_np])
# img_noisy_np.shape

# Define model

In [18]:
INPUT = 'noise' # 'meshgrid'
pad = 'reflection'
OPT_OVER = 'net' # 'net,input'

reg_noise_std = 1./30. # set to 1./20. for sigma=50
# 0.0001 0.001 0.01 0.1
LR = 0.01

OPTIMIZER='adam' # 'LBFGS'
# OPTIMIZER='LBFGS'
show_every = 100
exp_weight=0.99

In [19]:
# 3000 4000 6000 10000 15000
num_iter = 2000
input_depth = 32 
figsize = 4 

In [20]:
# returns a model
# net = get_net(input_depth, 'UNet', pad,
#               skip_n33d=128, 
#               skip_n33u=128, 
#               skip_n11=4, 
#               num_scales=5,
              # upsample_mode='bilinear').type(dtype)
net = get_net(input_depth, 'skip', pad,
              skip_n33d=128, 
              skip_n33u=128, 
              skip_n11=4, 
              num_scales=5,
              n_channels=1,
              upsample_mode='bilinear').type(dtype)

inside skip


In [21]:
# ./utils/common_utils.py
net_input = get_noise(input_depth, INPUT, (img_pil.size[1], img_pil.size[0]), noise_type='n').type(dtype).detach()
net_input.shape

torch.Size([1, 32, 672, 672])

In [22]:
# Compute number of parameters
s  = sum([np.prod(list(p.size())) for p in net.parameters()]); 
print ('Number of params: %d' % s)

Number of params: 2217573


In [23]:
# Loss
mse = torch.nn.MSELoss().type(dtype)
# noise to change and see
img_noisy_torch = np_to_torch(img_noisy_np).type(dtype)

# Optimize Model

In [24]:
net_input_saved = net_input.detach().clone()
noise = net_input.detach().clone()
out_avg = None
last_net = None
psrn_noisy_last = 0

i = 0

In [25]:
# Loss funftion printing image
def closure():
    
    global i, out_avg, psrn_noisy_last, last_net, net_input
    
    if reg_noise_std > 0:
        net_input = net_input_saved + (noise.normal_() * reg_noise_std)
    
    out = net(net_input)
    
    # Smoothing
    if out_avg is None:
        out_avg = out.detach()
    else:
        out_avg = out_avg * exp_weight + out.detach() * (1 - exp_weight)
            
    total_loss = mse(out, img_noisy_torch)
    total_loss.backward()
        
    # print(img_noisy_np.shape, out.detach().cpu().numpy()[0].shape)
    psrn_noisy = compare_psnr(img_noisy_np, out.detach().cpu().numpy()[0]) 
    psrn_gt    = compare_psnr(img_np, out.detach().cpu().numpy()[0]) 
    psrn_gt_sm = compare_psnr(img_np, out_avg.detach().cpu().numpy()[0]) 
    
    # So 'PSRN_gt', 'PSNR_gt_sm' only when ground truth is provided
    print('Iteration %05d    Loss %f   PSNR_noisy: %f   PSRN_gt: %f PSNR_gt_sm: %f' % (i, total_loss.item(), psrn_noisy, psrn_gt, psrn_gt_sm), end='')

    if  PLOT and i % show_every == 0:
        out_np = torch_to_np(out)
        plot_image_grid([np.clip(out_np, 0, 1), 
                         np.clip(torch_to_np(out_avg), 0, 1)], factor=figsize, nrow=1)
        
         
    
    # Backtracking
    if i % show_every:
        if psrn_noisy - psrn_noisy_last < -5: 
            print('Falling back to previous checkpoint.')

            for new_param, net_param in zip(last_net, net.parameters()):
                net_param.data.copy_(new_param.cuda())

            return total_loss*0
        else:
            last_net = [x.detach().cpu() for x in net.parameters()]
            psrn_noisy_last = psrn_noisy
            
    i += 1

    return total_loss

In [26]:
p = get_params(OPT_OVER, net, net_input)

In [38]:
optimize(OPTIMIZER, p, closure, LR, num_iter)

In [28]:
out_np = torch_to_np(net(net_input))

In [39]:
q = plot_image_grid([np.clip(out_np, 0, 1), img_np], factor=13);

#----------------------

In [30]:
(img_pil.size[1], img_pil.size[0])

(672, 672)

In [31]:
zero  = torch.zeros([1 ,32 ,544 ,896])
var=1./10

In [32]:
res = zero.normal_()*var
res<0

tensor([[[[ True,  True, False,  ...,  True, False, False],
          [ True, False, False,  ...,  True,  True, False],
          [ True,  True, False,  ...,  True, False, False],
          ...,
          [False, False, False,  ...,  True,  True, False],
          [ True, False,  True,  ...,  True,  True,  True],
          [False,  True, False,  ...,  True, False, False]],

         [[ True, False,  True,  ..., False, False,  True],
          [False, False, False,  ...,  True,  True, False],
          [ True, False,  True,  ...,  True, False,  True],
          ...,
          [False,  True,  True,  ..., False, False, False],
          [False, False, False,  ..., False, False,  True],
          [False,  True,  True,  ...,  True, False,  True]],

         [[False, False, False,  ...,  True, False, False],
          [False, False,  True,  ..., False,  True, False],
          [False,  True, False,  ...,  True, False, False],
          ...,
          [ True, False, False,  ..., False,  True,

In [33]:
img_noisy_np.shape

(1, 672, 672)

In [34]:
torch.zeros([4,2,3])

tensor([[[0., 0., 0.],
         [0., 0., 0.]],

        [[0., 0., 0.],
         [0., 0., 0.]],

        [[0., 0., 0.],
         [0., 0., 0.]],

        [[0., 0., 0.],
         [0., 0., 0.]]])

In [35]:
loc, scale = 0., 1.
s = np.random.laplace(loc, scale, [2,5,8])

In [36]:
# np_to_torch(s).shape
torch.from_numpy(s).shape

torch.Size([2, 5, 8])